In [1]:
import numpy as np
from typing import List
from dataclasses import dataclass

# @dataclass
# class Chromosome:
#     gene : List[int]
#     phenome : float = 
#     length : int = 0
#     distnace_metric : List[List[int]]

#     def __post_init__(self):
#         self.length = len(self.gene)
#         self.phenome = fitness(self.gene,distance)
    
    

In [2]:
from version2.classes import Chromosome, Parameters
from version2.crossover import point_crossover , ordered_crossover
from version2.mutation import mutate, multi_mutate, inversion, scramble
from version2.replacement import replace_firstweak , replace_weakest
from version2.utils import load_metrics, generate_population, parents_selection, create_chromosome, best_gene, pop_stats


set xml path

In [3]:
burma_path = './dataset/burma14.xml'
brazil_path = './dataset/brazil58.xml'

burma_metric = load_metrics(burma_path)
brazil_path = load_metrics(brazil_path)

d:\USERS\Exeter\Courses\Module ECMM409 (2023) Nature-Inspired Computation\Nature_inspired\project01\version2\utils.py:17: RuntimeWarning: invalid value encountered in multiply
  weights_metric = np.nan_to_num(np.identity(num_node) * -np.inf, 0.0)


In [4]:
import torch.nn as nn

class GENETIC(nn.Module):
    def __init__(self, distance_metric) -> None:
        super().__init__()
        self.distance_metric = distance_metric
        
    def forward(self, x):
        pass

In [5]:
@dataclass
class Result:
    last_population : List[Chromosome]
    best_candidate : Chromosome
    avg_fitness : List[float]
    max_fitness : List[float]
    stop_generation : int

In [6]:
from typing import Callable, List, Any

def search(distance_metric: List[List[int]],
        max_gens:int,
        pop_size:int,
        tour_size:int, 
        co_fn:Callable,
        mut_fn:Callable,
        replace_fn:Callable) -> Any:
    # population = generate_population(distance_metric=sample, pop_size=pop_size)
    population = generate_population(population_size=pop_size,
                                     distance_metric=distance_metric)
    avg_fitness = [np.mean([pop.phenome for pop in population])]
    max_fitness = [np.max([pop.phenome for pop in population])]
    best_candidate = best_gene(population=population)
    # best_candidate:Chromosome = population[np.argmax([pop.phenome for pop in population])]
    for i in range(max_gens):
        # parent1 = tournament_selection(population=population, fitness_value=pop_fitness, tournament_size=tour_size)
        # parent1, parent2 = binary_tournament(population=population, fitness_value=pop_fitness, tournament_size=tour_size)
        parents = parents_selection(population=population, tournament_size=tour_size)
        parent1 , parent2 = parents[0], parents[1]
        gene1, gene2 = co_fn(parent1,parent2)
        child1 , child2 = create_chromosome(gene1,distance_metric=distance_metric), create_chromosome(gene2,distance_metric=distance_metric)
        child1, child2 = mut_fn(child1), mut_fn(child2)
        population = replace_fn(child1)
        population = replace_fn(child2)
        stats = pop_stats(population=population)
        avg_fitness.append(stats[0])
        max_fitness.append(stats[1][1])
        new_best_candidate = best_gene(population=population)

        if new_best_candidate.phenome > best_candidate.phenome:
            best_candidate = new_best_candidate
        if i % 100:
            print("Processing {i} Steps")
        
        # child1 , child2 = co_fn(parent1=parent1, parent2=parent2)
        # child1 , child2 = mut_fn(child1) , mut_fn(child2)
        # population = replace_firstweak(population=population,candidate=child1,distance_metric=sample)
        # population = replace_firstweak(population=population,candidate=child2,distance_metric=sample)
        # pop_fitness = [fitness(pop, distance_metric=sample) for pop in population]
        # fit_avg.append(np.mean(pop_fitness))
        # fit_upper.append(np.max(pop_fitness))
        if (i > 2) and (avg_fitness[i] - avg_fitness[i-1] < 1e-5):
            return population , avg_fitness , max_fitness , i ,best_candidate

    return population , avg_fitness , max_fitness , i ,best_candidate